In [79]:
import KEGGutils as kg
import networkx as nx
from pyvis.network import Network

In [54]:
pathway = kg.KEGGpathway(pathway_id="hsa05215")
pathway.title

'Prostate cancer'

In [55]:
def element_repr(element: dict):
    return "\n".join(f"{key}: {value}" for key, value in element.items())

In [87]:
nt = Network(directed=True, width="100%", height="100%")

for node_id, node_data in pathway.nodes.data():
    nt.add_node(node_id, label=node_data["name"], title=element_repr(node_data))

for edge_id, edge_data in pathway.relations.items():
    nt.add_edge(edge_data["node_ids"][0], edge_data["node_ids"][1], title=element_repr(edge_data))

In [88]:
nt.set_edge_smooth('dynamic')
nt.toggle_physics(True)
nt.show_buttons(filter_=['physics'])
nt.show("pathway.html")